In [27]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
import config

In [28]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [29]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [30]:
results = sp.search(q="Like a Rolling Stone",limit=1,market="GB")
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Like+a+Rolling+Stone&type=track&market=GB&offset=0&limit=1',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/74ASZWbe4lXaubB36ztrGX'},
       'href': 'https://api.spotify.com/v1/artists/74ASZWbe4lXaubB36ztrGX',
       'id': '74ASZWbe4lXaubB36ztrGX',
       'name': 'Bob Dylan',
       'type': 'artist',
       'uri': 'spotify:artist:74ASZWbe4lXaubB36ztrGX'}],
     'external_urls': {'spotify': 'https://open.spotify.com/album/6YabPKtZAjxwyWbuO9p4ZD'},
     'href': 'https://api.spotify.com/v1/albums/6YabPKtZAjxwyWbuO9p4ZD',
     'id': '6YabPKtZAjxwyWbuO9p4ZD',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b27341720ef0ae31e10d39e43ca2',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e0241720ef0ae31e10d39e43ca2',
       'width': 300},
      {'height': 64,
       'url': 'h

In [31]:
Top1000 = sp.user_playlist_tracks("spotify", "2iBH9S3UXlrtUBxjffgZEh")

In [32]:
Top1000['items']

[{'added_at': '2014-07-14T02:20:12Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/1269437965'},
   'href': 'https://api.spotify.com/v1/users/1269437965',
   'id': '1269437965',
   'type': 'user',
   'uri': 'spotify:user:1269437965'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'compilation',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3eqjTLE0HfPfh78zjh6TqT'},
      'href': 'https://api.spotify.com/v1/artists/3eqjTLE0HfPfh78zjh6TqT',
      'id': '3eqjTLE0HfPfh78zjh6TqT',
      'name': 'Bruce Springsteen',
      'type': 'artist',
      'uri': 'spotify:artist:3eqjTLE0HfPfh78zjh6TqT'}],
    'available_markets': [],
    'external_urls': {'spotify': 'https://open.spotify.com/album/6AoxeGYGY2qSi8DwUQoGdx'},
    'href': 'https://api.spotify.com/v1/albums/6AoxeGYGY2qSi8DwUQoGdx',
    'id': '6AoxeGYGY2qSi8DwUQoGdx',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d

In [33]:
playlist = Top1000
print(list(Top1000.keys())) #look at items and total:
print("Total number of songs in the playlist: ",Top1000["total"]) #  Let's check items:
len(Top1000["items"])

['href', 'items', 'limit', 'next', 'offset', 'previous', 'total']
Total number of songs in the playlist:  1660


100

In [34]:
#Top1000['items'][0]
#to fetch the first track's info

In [35]:
Top1000["items"][0]["track"]["name"] #find title of the first track

'Born in the U.S.A.'

In [36]:
'''
#how to get all the pages of the playlist in one 
playlist= sp.user_playlist_tracks("spotify", "2iBH9S3UXlrtUBxjffgZEh")
play2 = playlist['items']
while playlist['next']:
    playlist = sp.next(playlist)
    play2.extend(playlist['items'])
    
len(play2)
'''

'\n#how to get all the pages of the playlist in one \nplaylist= sp.user_playlist_tracks("spotify", "2iBH9S3UXlrtUBxjffgZEh")\nplay2 = playlist[\'items\']\nwhile playlist[\'next\']:\n    playlist = sp.next(playlist)\n    play2.extend(playlist[\'items\'])\n    \nlen(play2)\n'

In [37]:
'''
#how to skip errors 
song_list = []
song_features_list = []
lenght = len(play2)
for i in range(lenght):
    try:
        song = play2[i]['track']['name']
        song_list.append(song)
        track = sp.search(q=song)
        features = sp.audio_features(track["tracks"]["items"][0]["id"])
        song_features_list.append(features)
        print(song)
    except:
        print('ERROR')
'''

'\n#how to skip errors \nsong_list = []\nsong_features_list = []\nlenght = len(play2)\nfor i in range(lenght):\n    try:\n        song = play2[i][\'track\'][\'name\']\n        song_list.append(song)\n        track = sp.search(q=song)\n        features = sp.audio_features(track["tracks"]["items"][0]["id"])\n        song_features_list.append(features)\n        print(song)\n    except:\n        print(\'ERROR\')\n'

In [39]:
#define a function to extract the playlist
def PlaylistDatabase(code):
    try:
        playlist = sp.user_playlist_tracks("spotify", code)
        update_playlist = playlist['items']
        while playlist['next']:
            update_playlist.extend(playlist['items'])
            playlist = sp.next(playlist)
        lenght = len(update_playlist)
        lenght
        song_features_list = []
        for i in range(lenght):
            try:
                song = update_playlist[i]['track']['name']
                #song_list.append(song)
                track = sp.search(q=song)
                features = sp.audio_features(track["tracks"]["items"][0]["id"])
                song_features_list.append(features)
                #print(len(song_features_list))
                if i % 50 == 0:
                    print(i, song)
            except:
                print('ERROR', i)
        df = pd.DataFrame()
        n_songs = len(song_features_list)
        for i in range(0, n_songs):
            #connect more track features in same playlist with concat
            df = pd.concat([df, pd.DataFrame(song_features_list[i])])
        df.reset_index(inplace=True, drop=True)
        df = df[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
                'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
        return df
    except:
        print('NOT WORKING')

In [40]:
print(len(playlist["items"]))

100


In [41]:
df = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'])
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature


In [42]:
df.reset_index(inplace=True, drop=True)
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature


In [43]:
df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [44]:
''''
#remove columns with categorical feature
df = df[['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
        'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
''''

SyntaxError: EOL while scanning string literal (1120348033.py, line 5)

In [45]:
#create dataframe with more than one playlist

In [46]:
#df.to_csv("pulpkoki.csv", encoding='utf-8')

In [48]:
playlist_list = ['1c5JpH8s61Se836y57iJpv']

In [49]:
df = pd.DataFrame()
for i in range(len(playlist_list)):
    newdf = PlaylistDatabase(playlist_list[i])
    df = pd.concat([df, newdf])

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'The Art Of Fugue, BWV 1080 - Later Version Of The Original Print Edited By Christoph Wolff: Contrapunctus 9. a 4 alla Duodecima', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


0 Hungarian Dance No.5 In G Minor, WoO 1 No. 5
ERROR 1


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Estancia / Danzas del Ballet: 2. Danza del trigo - Live At Centro de Acción Social por la Música, Sala Simón Bolivar, Caracas / 2008', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


ERROR 24
ERROR 29
ERROR 47
50 Sonata For Violin And Piano No.8 In G, Op.30 No.3: 3. Allegro vivace
ERROR 64
ERROR 67
ERROR 78
ERROR 93


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'The Art Of Fugue, BWV 1080 - Later Version Of The Original Print Edited By Christoph Wolff: Contrapunctus 9. a 4 alla Duodecima', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


...

600 Concerto For Violin And Strings In E, Op.8, No.1, RV 269 "La Primavera": 1. Allegro
ERROR 601
ERROR 602


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Concerto For Violin And Strings In G Minor, Op.8, No.2, R.315 "L\'estate": 1. Allegro non molto - Allegro', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Concerto For Violin And Strings In G Minor, Op.8, No.2, RV 315 "L\'estate": 2. Adagio - Presto - Adagio', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Concerto For Violin And Strings In G Minor, Op.8, No.2, RV 315 "L\'estate": 3. Presto (Tempo impetuoso d\'estate)', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Concerto For Violin And Strings In F, Op.8, No.3, RV 293 "L\'autunno": 1. Allegro (Ballo, e cant

ERROR 603
ERROR 604
ERROR 605
ERROR 606


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'Concerto For Violin And Strings In F, Op.8, No.3, RV 293 "L\'autunno": 2. Adagio molto (Ubriachi dormienti)', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.


ERROR 607
650 Sonata In C Major, Kk.159: Allegro
ERROR 680
ERROR 682
ERROR 683
ERROR 684
700 Mass In B Minor, BWV 232 / Credo: Credo in unum Deum
750 Bredon Hill And Other Songs: Bredon Hill
ERROR 769
